In [1]:
!pip install datasets
!pip install transformers -U
!pip install accelerate -U
!pip install trl
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 138.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.0 MB/s eta 0:00:00


In [2]:
!pip install peft

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
from datasets import load_dataset

DATASET_NAME = "ChrisHayduk/Llama-2-SQL-Dataset"

dataset = load_dataset(DATASET_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/631 [00:00<?, ?B/s]

data/train-00000-of-00001-922416e34c5bc7(…):   0%|          | 0.00/9.11M [00:00<?, ?B/s]

data/eval-00000-of-00001-6907aec719559d7(…):   0%|          | 0.00/1.01M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/70719 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/7858 [00:00<?, ? examples/s]

In [5]:
full_training_dataset = dataset["train"]
shuffled = full_training_dataset.shuffle()
training_dataset = shuffled.select(range(1000))

In [6]:
import bitsandbytes as bnb
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4")


In [7]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "NousResearch/Llama-2-7b-hf"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

model.config.use_cache = True

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [8]:
def construct_datapoint(x):
  combined = x['input'] + x['output']
  return tokenizer(combined, padding = True)

training_dataset = training_dataset.map(construct_datapoint)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
print(training_dataset)

Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask'],
    num_rows: 1000
})


In [10]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['q_proj', 'k_proj', 'down_proj', 'v_proj', 'gate_proj', 'o_proj', 'up_proj'],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config) # all layers except attention are frozen

generation_config = model.generation_config
generation_config.max_new_tokens = 256
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.temperature = 0.7
generation_config.top_p = 0.9
generation_config.do_sample = True

In [11]:
def generate(prompt):
  generation_config.max_new_tokens = 20

  encoded = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to(device)
  with torch.inference_mode():
    output_tokens = model.generate(
        input_ids=encoded,
        generation_config=generation_config,
        repetition_penalty = 1.3
    )

  string_decoded = tokenizer.decode(output_tokens[0], clean_up_tokenization_spaces=True)
  print(string_decoded)

In [14]:
generate('today I want to')

<s> today I want to share a very special video with you. sierpina, which was filmed by me and my husband


In [15]:
training_arguements = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    optim = "paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    output_dir="fine_tuning"
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=training_dataset,
    args=training_arguements,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False

In [16]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sashi8a (sashi8a-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


TrainOutput(global_step=250, training_loss=0.6484623413085937, metrics={'train_runtime': 1612.2078, 'train_samples_per_second': 0.62, 'train_steps_per_second': 0.155, 'total_flos': 5064819555753984.0, 'train_loss': 0.6484623413085937, 'epoch': 1.0})

In [ ]:
#API Key: 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

In [19]:
eval_dataset = dataset['eval'].shuffle()

sample_sql_question = eval_dataset[0]['input']
correct_answer= eval_dataset[0]['output']

print('Query:',sample_sql_question)
print('Answer:',correct_answer)

Query: Below is an instruction that describes a SQL generation task, paired with an input that provides further context about the available table schemas. Write SQL code that appropriately answers the request.

### Instruction:
What is the result on November 1, 1992?

### Input:
CREATE TABLE table_name_53 (result VARCHAR, date VARCHAR)

### Response: 
Answer: SELECT result FROM table_name_53 WHERE date = "november 1, 1992"


In [20]:
generate(sample_sql_question)

<s> Below is an instruction that describes a SQL generation task, paired with an input that provides further context about the available table schemas. Write SQL code that appropriately answers the request.

### Instruction:
What is the result on November 1, 1992?

### Input:
CREATE TABLE table_name_53 (result VARCHAR, date VARCHAR)

### Response:  SELECT below Below Below Below Below Below Below Below Below Below Below Below Below Below Below Below Below Below Below
